In [1]:
# 🔄 Auto-reload modules when their source code changes
%load_ext autoreload
%autoreload 2

# 📦 Set up src/ and utils/ paths (project root is detected via pyproject.toml)
%run ../bootstrap.py

In [2]:
import requests
from datetime import datetime, timedelta, timezone
import json
from pathlib import Path
from dotenv import load_dotenv
import os
import time
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import display, clear_output

from sleeping_beauty.config.config import Config

In [3]:
load_dotenv()

True

In [4]:
config = Config()


PROJECT_ROOT = Path(config.PROJECT_ROOT)
config.config_path = str(PROJECT_ROOT / "configs" / "config.yaml")
# "configs/FE/features_config.yaml"
config.load_from_yaml(config.config_path)




[Config] Loaded YAML config: /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml


In [5]:
config.print_config_info()

📂 Configuration
--------------------------------------------------
Configuration file:       /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml
--------------------------------------------------
🔐 Auth / Oura
--------------------------------------------------
Token path:               /Users/kenneth/.sleeping_beauty/oura_token.json
Client ID set:            True
Client Secret set:        True
Redirect URI:             http://localhost:8400/callback
Scopes:                   ['daily', 'heartrate', 'personal', 'session']


In [6]:
from datetime import date, timedelta
import asyncio

from sleeping_beauty.clients.oura_api_client import OuraApiClient
from sleeping_beauty.oura.auth.oura_auth import OuraAuth  
from sleeping_beauty.oura.auth.storage.file_storage import FileTokenStorage

In [ ]:
from sleeping_beauty.oura.auth.domain.auth_preflight_result import AuthPreflightReport


storage = FileTokenStorage(
    path=Path(
        config.oura_token_path
    ).expanduser()
)

# Instantiate your existing auth service
oura_auth = OuraAuth.from_config()
preflight: AuthPreflightReport = oura_auth.preflight_check()
print("\n".join(preflight.messages))

# Token provider callable (what the client expects)
token_provider = oura_auth.get_access_token   ## notice this is passing the get_access_token as the callable function 

# Create the API client
client = OuraApiClient(token_provider=token_provider)

🔐 Oura Auth Preflight Check
----------------------------------------
✅ client_id present
✅ client_secret present
✅ redirect_uri: http://localhost:8400/callback
✅ requested scopes: ['extapi:daily', 'extapi:heartrate', 'extapi:personal', 'extapi:session']
✅ token found in storage
✅ token valid for ~43180 minutes
✅ token covers requested scopes
----------------------------------------
✔️  Preflight check complete


In [8]:
# ---------------------------------------------------------------------
# Time window (keep small first — high volume endpoint)
# ---------------------------------------------------------------------
end_dt = datetime.now(timezone.utc).replace(microsecond=0)
start_dt = end_dt - timedelta(hours=6)


# ---------------------------------------------------------------------
# 1. Fetch a single page
# ---------------------------------------------------------------------
page = await client.get_heartrate_page(
    start_datetime=start_dt,
    end_datetime=end_dt,
)

print("=== First page ===")
print(f"Samples: {len(page.data)}")
print(f"Next token: {page.next_token!r}")

if page.data:
    sample = page.data[0]
    print("\nFirst sample:")
    print(sample)

# ---------------------------------------------------------------------
# 2. Iterate fully (small window only)
# ---------------------------------------------------------------------
print("\n=== Iterating all samples ===")

count = 0
async for sample in client.iter_heartrate(
    start_datetime=start_dt,
    end_datetime=end_dt,
):
    if count < 5:
        print(sample)
    count += 1

print(f"\nTotal samples iterated: {count}")

=== First page ===
Samples: 6
Next token: None

First sample:
HeartRateSample(bpm=61, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 19, 4, tzinfo=datetime.timezone.utc))

=== Iterating all samples ===
HeartRateSample(bpm=61, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 19, 4, tzinfo=datetime.timezone.utc))
HeartRateSample(bpm=62, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 24, 48, tzinfo=datetime.timezone.utc))
HeartRateSample(bpm=64, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 53, 14, tzinfo=datetime.timezone.utc))
HeartRateSample(bpm=63, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 53, 44, tzinfo=datetime.timezone.utc))
HeartRateSample(bpm=65, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 53, 48, tzinfo=datetime.timezone.utc))

Total samples iterated: 6


In [9]:
# ---------------------------------------------------------------------
# Time window (keep small first — high volume endpoint)
# ---------------------------------------------------------------------
from zoneinfo import ZoneInfo


tz = ZoneInfo("Europe/Luxembourg")

end_dt = datetime.now(tz).replace(microsecond=0)
start_dt = end_dt - timedelta(hours=6)

# ---------------------------------------------------------------------
# 1. Fetch a single page
# ---------------------------------------------------------------------
page = await client.get_heartrate_page(
    start_datetime=start_dt,
    end_datetime=end_dt,
)

print("=== First page ===")
print(f"Samples: {len(page.data)}")
print(f"Next token: {page.next_token!r}")

if page.data:
    sample = page.data[0]
    print("\nFirst sample:")
    print(sample)

# ---------------------------------------------------------------------
# 2. Iterate fully (small window only)
# ---------------------------------------------------------------------
print("\n=== Iterating all samples ===")

count = 0
async for sample in client.iter_heartrate(
    start_datetime=start_dt,
    end_datetime=end_dt,
):
    if count < 5:
        print(sample)
    count += 1

print(f"\nTotal samples iterated: {count}")

=== First page ===
Samples: 6
Next token: None

First sample:
HeartRateSample(bpm=61, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 19, 4, tzinfo=datetime.timezone.utc))

=== Iterating all samples ===
HeartRateSample(bpm=61, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 19, 4, tzinfo=datetime.timezone.utc))
HeartRateSample(bpm=62, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 24, 48, tzinfo=datetime.timezone.utc))
HeartRateSample(bpm=64, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 53, 14, tzinfo=datetime.timezone.utc))
HeartRateSample(bpm=63, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 53, 44, tzinfo=datetime.timezone.utc))
HeartRateSample(bpm=65, source='awake', timestamp=datetime.datetime(2026, 1, 11, 3, 53, 48, tzinfo=datetime.timezone.utc))

Total samples iterated: 6
